### Setup & load data

In [1]:
import graphlab
song_data = graphlab.SFrame('song_data.gl/')
graphlab.canvas.set_target('ipynb')

This non-commercial license of GraphLab Create for academic use is assigned to mmavricek@burning-glass.com and will expire on October 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\MMAVRI~1.BGT\AppData\Local\Temp\graphlab_server_1510089156.log.0


### Create training and test data

In [2]:
train_data, test_data = song_data.random_split(.8, seed = 0)

In [3]:
song_data.head(5)

Columns:
	user_id	str
	song_id	str
	listen_count	int
	title	str
	artist	str
	song	str

Rows: 5

Data:
+-------------------------------+--------------------+--------------+
|            user_id            |      song_id       | listen_count |
+-------------------------------+--------------------+--------------+
| b80344d063b5ccb3212f76538f... | SOAKIMP12A8C130995 |      1       |
| b80344d063b5ccb3212f76538f... | SOBBMDR12A8C13253B |      2       |
| b80344d063b5ccb3212f76538f... | SOBXHDL12A81C204C0 |      1       |
| b80344d063b5ccb3212f76538f... | SOBYHAJ12A6701BF1D |      1       |
| b80344d063b5ccb3212f76538f... | SODACBL12A8C13C273 |      1       |
+-------------------------------+--------------------+--------------+
+-----------------+---------------+-------------------------------+
|      title      |     artist    |              song             |
+-----------------+---------------+-------------------------------+
|     The Cove    |  Jack Johnson |    The Cove - Jack Johnson    |
| Entre Dos Aguas | Paco De Lucia | Entre Dos Aguas - Paco De ... |
|     Stronger    |   Kanye West  |     Stronger - Kanye West     |
|  Constellations |  Jack Johnson | Constellations - Jack Johnson |
|   Learn To Fly  |  Foo Fighters |  Learn To Fly - Foo Fighters  |
+-----------------+---------------+-------------------------------+
[5 rows x 6 columns]

# Q1: Counting Unique Users
The method .unique() can be used to select the unique elements in a column of data. In this question, you will compute the number of unique users who have listened to songs by various artists. For example, to find out the number of unique users who listened to songs by 'Kanye West', all you need to do is select the rows of the song data where the artist is 'Kanye West', and then count the number of unique entries in the ‘user_id’ column. 

Compute the number of unique users for each of these artists: 'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.

In [5]:
kw_count = len(song_data[song_data['artist'] == 'Kanye West']['user_id'].unique())
ff_count = len(song_data[song_data['artist'] == 'Foo Fighters']['user_id'].unique())
ts_count = len(song_data[song_data['artist'] == 'Taylor Swift']['user_id'].unique())
lg_count = len(song_data[song_data['artist'] == 'Lady GaGa']['user_id'].unique())
print 'Unique users that listened to Kanye: %s' % kw_count
print 'Unique users that listened to Foo Fighters: %s' % ff_count
print 'Unique users that listened to Taylor Swift: %s' % ts_count
print 'Unique users that listened to Lady GaGa: %s' % lg_count

Unique users that listened to Kanye: 2522
Unique users that listened to Foo Fighters: 2055
Unique users that listened to Taylor Swift: 3246
Unique users that listened to Lady GaGa: 2928


# Q2: Most & Least Popular Artists

Using groupby-aggregate to find the most popular and least popular artist: each row of song_data contains the number of times a user listened to particular song by a particular artist. If we would like to know how many times any song by 'Kanye West' was listened to, we need to select all the rows where ‘artist’=='Kanye West' and sum the ‘listen_count’ column. 

In [8]:
summ_artist = song_data.groupby(key_columns = 'artist', operations = {'total_count': graphlab.aggregate.SUM('listen_count')})
summ_artist = summ_artist.sort('total_count', ascending = False)
print summ_artist.head(5)
print summ_artist.tail(5)

+------------------------+-------------+
|         artist         | total_count |
+------------------------+-------------+
|     Kings Of Leon      |    43218    |
|     Dwight Yoakam      |    40619    |
|         Björk          |    38889    |
|        Coldplay        |    35362    |
| Florence + The Machine |    33387    |
+------------------------+-------------+
[5 rows x 2 columns]

+-------------------------------+-------------+
|             artist            | total_count |
+-------------------------------+-------------+
|             Diplo             |      30     |
|         Boggle Karaoke        |      30     |
| Beyoncé feat. Bun B and Sl... |      26     |
|         Reel Feelings         |      24     |
|        William Tabbert        |      14     |
+-------------------------------+-------------+
[5 rows x 2 columns]



# Q3:  [OPTIONAL] Using groupby-aggregate to find the most recommended songs

In [9]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                                user_id = 'user_id',
                                                                item_id = 'song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 1.85s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.94541s

We are going to make recommendations for the users in the test data, but there are over 200,000 users (58,628 unique users) in the test set. Computing recommendations for these many users can be slow in some computers. Thus, we will use only the first 10,000 users only in this question. Using this command to select this subset of users:

In [10]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [12]:
# Compute one recommended song for each of these test users
top_song = personalized_model.recommend(subset_test_users, k=1

recommendations finished on 1000/10000 queries. users per second: 11767.1

recommendations finished on 2000/10000 queries. users per second: 12905.8

recommendations finished on 3000/10000 queries. users per second: 13455.6

recommendations finished on 4000/10000 queries. users per second: 13701.4

recommendations finished on 5000/10000 queries. users per second: 14207.4

recommendations finished on 6000/10000 queries. users per second: 14460.7

recommendations finished on 7000/10000 queries. users per second: 14555.9

recommendations finished on 8000/10000 queries. users per second: 14845.3

recommendations finished on 9000/10000 queries. users per second: 14928.3

recommendations finished on 10000/10000 queries. users per second: 14861.8

In [13]:
top_song.head(5)

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504536115,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424376488,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.0238929539919,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.017026577677,1
3a7111f4cdf3c5a85fd4053e3cc2333562e1e0cb ...,Heartbreak Warfare - JohnMayer ...,0.0298416515191,1


In [14]:
summ_song = top_song.groupby(key_columns='song', 
                             operations={'count': graphlab.aggregate.COUNT()}).sort('count', ascending = False)
summ_song.head(5)

song,count
Undo - Björk,436
Secrets - OneRepublic,384
Revelry - Kings Of Leon,225
You're The One - DwightYoakam ...,163
Fireflies - CharttraxxKaraoke ...,123
